In [1]:
import os
import re
import folium
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm.notebook import tqdm

def listdir_nohidden(path):
    def ld_nh(path):
        for f in os.listdir(path):
            if not f.startswith('.'):
                yield f
    return list (ld_nh(path))

import geopandas as gp

### run the notebook once for each election

In [3]:
data_dir = '../raw_turnout_by_precinct'
elections = [e for e in listdir_nohidden(data_dir) if '.' not in e ]
elections

['2018_november', '2016_may', '2018_may']

#### (cycle this index 0-2 through 2:)

In [40]:
e_idx = 2
election = elections[e_idx]

target_dir = f'../{election.lower()}_cleaned'

if target_dir[3:] not in os.listdir('..'):
    os.makedirs(target_dir)
print(target_dir)

../2018_may_cleaned


### identify data for both races (`init` + `runoff`)

In [41]:
e_races = listdir_nohidden(os.path.join(data_dir, election )) 


# if parties are separated
if 'DEM' in os.listdir(os.path.join(data_dir, election, e_races[0])):
    party = 'DEM'
else:
    party = ''

init_race = [c+'/'+party for c in e_races if 'Runoff' not in c][0]
runoff_race = [c+'/'+party for c in e_races if 'Runoff' in c][0]



init_race_files = listdir_nohidden(os.path.join(data_dir, election, init_race )) 
runoff_race_files = listdir_nohidden(os.path.join(data_dir, election, runoff_race )) 

try: assert init_race_files == runoff_race_files
except:
    print('Initial election data does not match final elections data')
    init_only = [i for i in init_race_files if i not in runoff_race_files]
    runoff_only = [i for i in runoff_race_files if i not in init_race_files]
    print('Missing from initial data:', runoff_only)
    print('Missing from runoffs data:', init_only)

e_races = {'INIT':
              {'name': init_race,
               'files': init_race_files,},
          
          'RUNOFF': 
              {'name': runoff_race,
               'files': runoff_race_files,},}

[race['name'] for race in e_races.values()]

['May 2018/DEM', 'June 2018 (Runoff)/DEM']

### parse data for one county in this election

In [42]:
def parse_pair_data(county):
    out = []
    for race_type, race in e_races.items(): # iterate county data for both INIT & RUNOFF

        county_file = os.path.join(data_dir, election, race['name'], county )

        data = pd.read_excel(county_file)

        #election_date = data.columns[0].split('\n')[-1].split(':')[1]

        # quick clean and specify columns
        data.loc[0] = data.T[0].fillna(method='ffill')
        data.loc[1] = data.loc[1]

        data.columns = (data.loc[1] + '_' + data.loc[0]
                       ).apply(
                            lambda x: 
                               str(x).replace('Reg #', 'N_registered'
                                    ).replace('Voted', 'N_voted'
                                    ).replace('%', 'PCT_voted'
                                    ).replace(' ', '_')
                                if str(x)!='nan' else ''                             
                            )
            
        data = data.rename(columns={k: race_type+'_'+k for k in data.columns})

        
        
        # grab index names from subsequent row & consolidate
        data.columns = list(data.loc[0].values[:2]) + list(data.columns)[2:]
        data = data[2:].reset_index(drop=True)

        out.append(data)
    
    out = pd.concat(out, axis=1)
    
    out['County'] = county.split('.')[0]
    
    # calc participation rate change
    out = out[['County']+[c for c in out.columns if c!='County']]
    return out

out = parse_pair_data(sorted(init_race_files)[0])

### combine data for one election (init + runoff)

In [43]:
e_data = pd.DataFrame()
for county in tqdm(init_race_files):
    out = parse_pair_data(county)
    e_data = pd.concat([e_data, out])

# drop miscellaneous & unknown parcels
e_data = e_data.drop_duplicates(subset=['County', 'PRECINCT ID']
                      ).dropna(subset=['PRECINCT ID'])
# drop duplicate columns
e_data = e_data.loc[:,~e_data.columns.duplicated()]

# drop (absentees?) 
e_data = e_data[~e_data['PRECINCT ID'].str.contains('88888')]
e_data = e_data[~e_data['PRECINCT ID'].str.contains('99999')]
e_data = e_data.reset_index(drop=True)

e_data = e_data.sort_values('PRECINCT ID').sort_values('County')

# engineer sums for specific groups

In [44]:
for c in e_data.columns:
    e_data = e_data.rename(columns={c:c.replace('TOTAL_VOTERS', 'ALL_ALL')})
e_data.head()

County PRECINCT ID PRECINCT DESCRIPTION INIT_N_registered_BLACK_MALE  \
1572  APPLING          5B                   5B                           49   
1564  APPLING          1B                   1B                           31   
1567  APPLING         3A1                  3A1                            6   
1565  APPLING          1C                   1C                            8   
1570  APPLING          4D                   4D                           28   

     INIT_N_voted_BLACK_MALE INIT_PCT_voted_BLACK_MALE  \
1572                       3                      6.12   
1564                       5                     16.13   
1567                       1                     16.67   
1565                       1                     12.50   
1570                       7                     25.00   

     INIT_N_registered_BLACK_FEMALE INIT_N_voted_BLACK_FEMALE  \
1572                             62                         5   
1564                             42                        10   
1567                              8                         2   
1565                             18                         0   
1570                             30                         9   

     INIT_PCT_voted_BLACK_FEMALE INIT_N_registered_BLACK_UNKNOWN  ...  \
1572                        8.06                               0  ...   
1564                       23.81                               0  ...   
1567                          25                               0  ...   
1565                           0                               0  ...   
1570                          30                               0  ...   

     RUNOFF_PCT_voted_UNKNOWN_MALE RUNOFF_N_registered_UNKNOWN_FEMALE  \
1572                          3.12                                 27   
1564                             0                                 27   
1567                             0                                 19   
1565                             0                                 29   
1570                             0                                 45   

     RUNOFF_N_voted_UNKNOWN_FEMALE RUNOFF_PCT_voted_UNKNOWN_FEMALE  \
1572                             0                               0   
1564                             1                             3.7   
1567                             0                               0   
1565                             0                               0   
1570                             0                               0   

     RUNOFF_N_registered_UNKNOWN RUNOFF_N_voted_UNKNOWN  \
1572                           0                      0   
1564                           0                      0   
1567                           0                      0   
1565                           0                      0   
1570                           0                      0   

     RUNOFF_PCT_voted_UNKNOWN RUNOFF_N_registered_ALL_ALL  \
1572                        0                        1078   
1564                        0                        1244   
1567                        0                         762   
1565                        0                         812   
1570                        0                        1420   

     RUNOFF_N_voted_ALL_ALL RUNOFF_PCT_voted_ALL_ALL  
1572                      4                     0.37  
1564                     11                     0.88  
1567                      5                     0.66  
1565                      5                     0.62  
1570                      5                     0.35  

[5 rows x 135 columns]

### create dataframe of group identity column keys 

In [45]:
ppl_stats = [c for c in e_data.columns if 'MALE' in c] # captures all people (interesting)

ppl_stat_dict = {} # compile list of columns relevant to each parent group type
for colname in ppl_stats:
    cparts = colname.split('_')
    ppl_stat_dict[colname] = {
        'sex': 'FEMALE' if 'FEMALE' in cparts[-1] else 'MALE',
        'race': cparts[-2]   } # get dummy descriptions for both categories in each column
    
ppl_cols_df = pd.DataFrame(ppl_stat_dict
    ).T.reset_index().rename(columns={'index':'col_name'})

print('Sexes:', ppl_cols_df.sex.unique())
print('Races:', ppl_cols_df.race.unique())

# create column groups for each parent person group

group_agg_map = {}

for p_idx in ppl_cols_df.index:    
    p_row = ppl_cols_df.loc[p_idx]
    
    if p_row.sex not in group_agg_map.keys():
        group_agg_map[p_row.sex] = []
    group_agg_map[p_row.sex].append(p_row.col_name)
    
    if p_row.race not in group_agg_map.keys():
        group_agg_map[p_row.race] = []
    group_agg_map[p_row.race].append(p_row.col_name)

print(len(group_agg_map), 'total identity groups defined')

# create custom super all-all category
group_agg_map['ALL'] = ppl_stats

Sexes: ['MALE' 'FEMALE']
Races: ['BLACK' 'WHITE' 'ASIA-PI' 'HISP-LT' 'NATIVE-AM' 'OTHER' 'UNKNOWN']
9 total identity groups defined


In [46]:
# function to get the sum total for each statistic in a parent identity group
def calc_group_sum(group):
    
    # id all cols for this group, drop PCT (all just calced.)
    target_cols = [c for c in group_agg_map[group] if 'pct' not in c.lower()]
    
    # group cols by stat type
    stat_cats = list(sorted(set(
        ['_'.join(c.split('_')[:-2]) for c in target_cols])))
    # the above are be universal ( [INIT/R.O.] + [REG/VOTED] )    
    
    # for the given group....
    stat_cols = {}
    for stat in stat_cats: # iterate to get sum of each statistic type
        
        stat_cols[stat] = [] # to compile relevant columns for this statistic
        for col in target_cols: 
            if stat in col: # identify target cols for THIS STAT
                stat_cols[stat].append(col) 
        
        if 'MALE' in group:
            sum_col_name = stat+'_ALL_'+group
        else:
            sum_col_name = stat+'_'+group+'_ALL' 
        
        
        e_data[sum_col_name] = e_data[ # this is where the addition occurs
                                stat_cols[stat]  # columns to add
                                     ].astype(float).sum(axis=1) # ADD ACROSS AXIS 1  
        
for group in group_agg_map.keys():
    calc_group_sum(group)

## create groupby'd version

In [47]:
for c in e_data.columns:
    try: e_data[c] = e_data[c].astype(float)
    except: pass
    
g_data = e_data.groupby('County').sum()

## calc delta & pct change stats for all groups

In [48]:
elecs = ['INIT', 'RUNOFF']
stats = ['N_voted', 'N_registered']

get_group = lambda x: '_'.join(x.split('_')[3:])

# ALL groups (original and engineered sums)
all_group_names = set([get_group(c) for c in e_data.columns[3:]])

for group in sorted(all_group_names):
    
    target_cols = [c for c in e_data.columns if group == get_group(c)]
        
    for stat in stats:
        stat_cols = [c for c in target_cols if stat in c]        
        
        delta_colname = stat_cols[0].replace('INIT', 'Delta')
        e_data[delta_colname] = e_data[stat_cols[1]].astype(int) - e_data[stat_cols[0]].astype(int)
        g_data[delta_colname] = g_data[stat_cols[1]].astype(int) - g_data[stat_cols[0]].astype(int)

        
        pct_colname = stat_cols[0].replace('INIT', 'pctChange')
        e_data[pct_colname] = e_data[delta_colname] / e_data[stat_cols[0]].astype(int)
        g_data[pct_colname] = g_data[delta_colname] / g_data[stat_cols[0]].astype(int)
        

### clean up feature names a bit more 

In [49]:
cols = list(e_data.columns)
re_map = {}
for col in cols[3:]:
    c = col.split('_')
    elec = c[0].capitalize()
    if 'Pct' in elec:
        elec = 'pctChange'
    stat = c[1].lower()+c[2].capitalize()
    race = c[3].capitalize()
    if len(c)>=5: sex = c[4].capitalize()
    else: sex = 'Unknown'        
    re_map[col] = f"{elec}_{stat}_{race}_{sex}"

e_data = e_data.rename(columns=re_map)
g_data = g_data.rename(columns=re_map)

In [50]:
# FUNCTION FOR DISPLAYING / RETURN DATA FOR A TARGET GROUP

def get_group_data(df, race, sex, elec=None):
    target_cols = []
    o_cols = []
    for col in df:
        if col.count('_')<3:
            o_cols.append(col)
            continue
        c = col.split('_')
        if not race or c[2] == race:
            if not elec or c[0] == elec:
                if not sex or c[3] == sex:
                    target_cols.append(col)
                        
    return df[ o_cols + target_cols ]

### re-calc group participation rates

In [51]:
elecs = ['Init', 'Runoff']
stats = ['nVoted', 'nRegistered']
races = sorted(set([c.split('_')[2] for c  in g_data.columns[3:]]))
sexes = sorted(set([c.split('_')[3] for c  in g_data.columns[3:]]))

for sex in sexes:
    for race in races:
        if race=='All' and sex=='Unknown': continue # doesnt exist
        
        for elec in elecs:
            group_data = get_group_data(g_data, race, sex, elec)

            for stat in stats:
                group_data[[c for c in group_data.columns if any(s in c for s in stats) ]]

            cols = group_data.columns
            if not len(cols):
                print(race, sex, elec)
                out = group_data
                

            target_fname = f"{elec}_pctVoted_{race}_{sex}"
            g_data[target_fname] = round(100*(g_data[cols[1]] / g_data[cols[0]]), 2)
            e_data[target_fname] = round(100*(e_data[cols[1]] / e_data[cols[0]]), 2)

## create state totals dataframe

In [52]:
vote_dict = {
    'Init': {},
    'Runoff': {},  }
regi_dict = {
    'Init': {},
    'Runoff': {},  }

for sex in sexes:
    for race in races:
        if race=='All' and sex=='Unknown': continue # doesnt exist
        for elec in elecs:
            group_data = get_group_data(g_data, race, sex, elec)
            
            category = '_'.join(group_data.columns[1].split('_')[2:])
            
            n_votes = group_data[group_data.columns[1]].sum()
            n_reg = group_data[group_data.columns[0]].sum()

            vote_dict[str(elec)][category] = n_votes
            regi_dict[str(elec)][category] = n_reg
            
state_totals = pd.DataFrame(vote_dict)

# quick features
state_totals = state_totals.rename(columns=
                {c:c.lower()+'Votes' for c in state_totals.columns})

# add n_registered back on
r_data = pd.DataFrame(regi_dict)
r_data = r_data.rename(columns=
                {c:c.lower()+'Registered' for c in r_data.columns})
state_totals = pd.merge(state_totals, r_data, left_index=True, right_index=True)

state_totals['initParticipation'] = round(100*(state_totals['initVotes'] / state_totals['initRegistered']), 2)
state_totals['runoffParticipation'] = round(100*(state_totals['runoffVotes'] / state_totals['runoffRegistered']), 2)

# sort columns for init then runoff
state_totals = state_totals[sorted(state_totals.columns)]



# CALCULATED FEATURES
state_totals['nVotesChange'] = state_totals['runoffVotes'] - state_totals['initVotes']


state_totals['ParticipationChange'] = round(100*(state_totals['nVotesChange'] / state_totals['initVotes']), 2)

# sort by group participation dropoff
#state_totals = state_totals.sort_values('ParticipationChange')

# OPTIONAL: ADD COMMA SEPARATORS FOR THOUSANDS (no longer numeric after this obviously)
for c in state_totals.columns:
    if 'Partic' not in c:
        state_totals[c] = state_totals.apply(lambda x:
                        "{:,}".format(int(x[c])) if str(x[c])!='nan' else '-', axis=1)
    else :
        state_totals[c] = state_totals.apply(lambda x:
                        "{}%".format(int(x[c]))  if str(x[c])!='nan' else '-', axis=1)

# CONVERT INDEX TO TWO SEPARATE COLUMNS
parse_race = lambda x: x.split('_')[0]
parse_sex = lambda x: x.split('_')[1]
state_totals['race'] = [parse_race(v) for v in state_totals.index]
state_totals['sex'] = [parse_sex(v) for v in state_totals.index]
cols = list(state_totals.columns)
state_totals = state_totals[cols[-2:]+cols[:-2]]

state_totals = state_totals.sort_values('sex')
state_totals = state_totals.sort_values('race')
state_totals = state_totals.replace('All', '(all)')

if party:
    fname = target_dir+f'/statewide_{party}_participation.csv'
else:
    fname = target_dir+'/statewide_participation.csv'
state_totals.to_csv(fname, index=False) # index is now redundant
print(fname, 'saved:')
state_totals

../2018_may_cleaned/statewide_DEM_participation.csv saved:


race      sex initParticipation initRegistered  \
All_All                (all)    (all)                8%      6,720,103   
All_Male               (all)     Male                6%      3,101,748   
All_Female             (all)   Female                9%      3,618,355   
Asia-pi_All          Asia-pi    (all)                4%        137,589   
Asia-pi_Unknown      Asia-pi  Unknown                4%            364   
Asia-pi_Male         Asia-pi     Male                4%         65,424   
Asia-pi_Female       Asia-pi   Female                5%         72,165   
Black_All              Black    (all)               16%      2,032,521   
Black_Unknown          Black  Unknown                5%          2,386   
Black_Male             Black     Male               13%        864,361   
Black_Female           Black   Female               18%      1,168,160   
Hisp-lt_Unknown      Hisp-lt  Unknown                2%            362   
Hisp-lt_Male         Hisp-lt     Male                4%         84,717   
Hisp-lt_Female       Hisp-lt   Female                5%        100,555   
Hisp-lt_All          Hisp-lt    (all)                4%        185,272   
Native-am_Female   Native-am   Female                4%          3,848   
Native-am_Unknown  Native-am  Unknown                0%             19   
Native-am_Male     Native-am     Male                3%          3,580   
Native-am_All      Native-am    (all)                3%          7,428   
Other_Unknown          Other  Unknown                3%            262   
Other_Male             Other     Male                5%         37,434   
Other_Female           Other   Female                8%         46,572   
Other_All              Other    (all)                7%         84,006   
Unknown_Male         Unknown     Male                4%        332,938   
Unknown_Female       Unknown   Female                6%        322,010   
Unknown_All          Unknown    (all)                5%        654,948   
Unknown_Unknown      Unknown  Unknown                4%          5,278   
White_All              White    (all)                4%      3,618,339   
White_Male             White     Male                4%      1,713,294   
White_Female           White   Female                5%      1,905,045   
White_Unknown          White  Unknown                3%          2,448   

                  initVotes runoffParticipation runoffRegistered runoffVotes  \
All_All             562,316                  2%        6,660,379     159,578   
All_Male            211,973                  1%        3,065,929      58,163   
All_Female          350,343                  2%        3,594,450     101,415   
Asia-pi_All           6,865                  1%          136,977       2,704   
Asia-pi_Unknown          17                  3%              364          13   
Asia-pi_Male          3,159                  2%           65,076       1,303   
Asia-pi_Female        3,706                  1%           71,901       1,401   
Black_All           338,476                  4%        2,013,986      95,185   
Black_Unknown           141                  1%            2,343          30   
Black_Male          119,091                  3%          852,254      31,986   
Black_Female        219,385                  5%        1,161,732      63,199   
Hisp-lt_Unknown           8                  0%              362           1   
Hisp-lt_Male          3,489                  0%           84,199         679   
Hisp-lt_Female        5,467                  1%          100,179       1,013   
Hisp-lt_All           8,956                  0%          184,378       1,692   
Native-am_Female        162                  1%            3,818          50   
Native-am_Unknown         0                  0%               19           0   
Native-am_Male          113                  0%            3,516          23   
Native-am_All           275                  1%            7,334          73   
Other_Unknown            10                  1%        

## save

In [53]:
# PRECINCT DATA
if party:
    elec_fpath = target_dir+f'/all_precincts_{party}_participation.csv'
else:
    elec_fpath = target_dir+'/all_precincts_participation.csv'
print('Saving precinct data...')
print('>', elec_fpath, '\n')
e_data.to_csv(elec_fpath, index=False)

# COUNTY DATA
if party:
    elec_fpath = target_dir+f'/all_counties_{party}_participation.csv'
else:
    elec_fpath = target_dir+'/all_counties_participation.csv'
print('Saving county data...')
print('>', elec_fpath)
g_data.to_csv(elec_fpath)

Saving precinct data...
> ../2018_may_cleaned/all_precincts_DEM_participation.csv 

Saving county data...
> ../2018_may_cleaned/all_counties_DEM_participation.csv
